https://wikidocs.net/30708

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
#시간별로 vote 수
popular=pd.read_csv("/content/drive/MyDrive/Github/netflix_EDA/data/popular_des2.csv",index_col=0)
no_popular = pd.read_csv("/content/drive/MyDrive/Github/netflix_EDA/data/non_popular_des2.csv",index_col=0)

In [3]:
popular[:5]

,description_x
1,"On a photo shoot in Ghana, an American model s..."
7,A rookie cop with one day to prove himself to ...
8,When a powerful businesswoman’s political ambi...
11,"Bored with being the Lord of Hell, the devil r..."
19,After growing up amidst the gang wars of his h...


In [4]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [5]:
import numpy as np

popular_list = popular.values.tolist()
no_popular_list = no_popular.values.tolist()
popular_list[:5]

[['On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.'],
 ["A rookie cop with one day to prove himself to a veteran LAPD narcotics officer receives a crash course in his mentor's questionable brand of justice."],
 ['When a powerful businesswoman’s political ambitions are threatened by her underworld connections, the ensuing power struggle could cost her everything.'],
 ['Bored with being the Lord of Hell, the devil relocates to Los Angeles, where he opens a nightclub and forms a connection with a homicide detective.'],
 ['After growing up amidst the gang wars of his hometown, Vincent forms an entrepreneurial squad of his own and ends up on the wrong side of the law.']]

In [6]:
import itertools
popular_list =list(itertools.chain.from_iterable(popular_list))
no_popular_list=list(itertools.chain.from_iterable(no_popular_list))

In [7]:
#텍스트 전처리
news_df = pd.DataFrame({'document':no_popular_list})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [8]:
news_df['clean_doc'][1]

'blackmailed company level employee finds himself forced boss rival former mentor'

In [9]:
import nltk
nltk.download('stopwords')
  
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
print(tokenized_doc[1])

['blackmailed', 'company', 'level', 'employee', 'finds', 'forced', 'boss', 'rival', 'former', 'mentor']


In [11]:
#3) TF-IDF 행렬 만들기
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [12]:
news_df['clean_doc'][1]

'blackmailed company level employee finds forced boss rival former mentor'

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(770, 1000)

In [14]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [15]:
np.shape(svd_model.components_)


(20, 1000)

In [16]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('young', 0.29159), ('life', 0.25955), ('woman', 0.24239), ('friends', 0.18003), ('family', 0.17523)]
Topic 2: [('friends', 0.39072), ('family', 0.2405), ('past', 0.19878), ('true', 0.1703), ('real', 0.16955)]
Topic 3: [('home', 0.4277), ('parents', 0.22191), ('couple', 0.20327), ('history', 0.18232), ('moves', 0.17275)]
Topic 4: [('world', 0.22884), ('high', 0.18752), ('mission', 0.18067), ('school', 0.1508), ('team', 0.1344)]
Topic 5: [('woman', 0.39847), ('relationship', 0.2172), ('tries', 0.18934), ('friends', 0.16511), ('break', 0.13831)]
Topic 6: [('friends', 0.30392), ('love', 0.27923), ('best', 0.1451), ('young', 0.13997), ('trip', 0.13484)]
Topic 7: [('life', 0.2957), ('tries', 0.15162), ('high', 0.15121), ('relationship', 0.14423), ('love', 0.14046)]
Topic 8: [('love', 0.36249), ('falls', 0.23681), ('couple', 0.22096), ('relationship', 0.18633), ('past', 0.1465)]
Topic 9: [('woman', 0.22963), ('world', 0.21191), ('mission', 0.20571), ('team', 0.14733), ('rescue', 0.

In [17]:
tokenized_doc[:5]

0    [arrival, charismatic, young, priest, brings, ...
1    [blackmailed, company, level, employee, finds,...
2    [hiking, trip, wild, turns, desperate, surviva...
3    [hiking, trip, wild, turns, desperate, surviva...
4    [status, strategy, collide, social, experiment...
Name: clean_doc, dtype: object

In [18]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0


[(16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)]


In [19]:
print(dictionary[66])

refugee


In [20]:
len(dictionary)

4012

In [21]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.010*"three" + 0.009*"young" + 0.009*"living" + 0.007*"friends"')
(1, '0.010*"young" + 0.008*"woman" + 0.007*"finds" + 0.007*"love"')
(2, '0.015*"high" + 0.006*"woman" + 0.006*"friends" + 0.006*"school"')
(3, '0.010*"group" + 0.008*"must" + 0.007*"couple" + 0.007*"find"')
(4, '0.007*"play" + 0.006*"young" + 0.006*"people" + 0.006*"killer"')
(5, '0.016*"young" + 0.007*"strange" + 0.005*"help" + 0.005*"friends"')
(6, '0.011*"friends" + 0.007*"life" + 0.006*"father" + 0.005*"relationship"')
(7, '0.007*"begins" + 0.007*"wife" + 0.007*"murdered" + 0.007*"dark"')
(8, '0.009*"four" + 0.008*"wife" + 0.007*"house" + 0.007*"young"')
(9, '0.008*"father" + 0.008*"must" + 0.008*"take" + 0.006*"along"')
(10, '0.012*"young" + 0.012*"woman" + 0.011*"life" + 0.006*"love"')
(11, '0.008*"mother" + 0.007*"former" + 0.006*"home" + 0.006*"family"')
(12, '0.013*"life" + 0.007*"mission" + 0.007*"remote" + 0.007*"falls"')
(13, '0.012*"find" + 0.007*"must" + 0.006*"love" + 0.006*"turns"')
(14, '0.010*"pas

In [22]:
print(ldamodel.print_topics())

[(0, '0.010*"three" + 0.009*"young" + 0.009*"living" + 0.007*"friends" + 0.007*"college" + 0.007*"find" + 0.006*"family" + 0.006*"crime" + 0.004*"girl" + 0.004*"decades"'), (1, '0.010*"young" + 0.008*"woman" + 0.007*"finds" + 0.007*"love" + 0.006*"story" + 0.006*"relationship" + 0.006*"terrorist" + 0.006*"world" + 0.004*"four" + 0.004*"teen"'), (2, '0.015*"high" + 0.006*"woman" + 0.006*"friends" + 0.006*"school" + 0.004*"become" + 0.004*"four" + 0.004*"world" + 0.004*"game" + 0.004*"tries" + 0.004*"university"'), (3, '0.010*"group" + 0.008*"must" + 0.007*"couple" + 0.007*"find" + 0.007*"woman" + 0.006*"parents" + 0.006*"young" + 0.006*"husband" + 0.006*"struggles" + 0.006*"fight"'), (4, '0.007*"play" + 0.006*"young" + 0.006*"people" + 0.006*"killer" + 0.005*"night" + 0.005*"love" + 0.005*"leader" + 0.005*"take" + 0.005*"time" + 0.005*"crime"'), (5, '0.016*"young" + 0.007*"strange" + 0.005*"help" + 0.005*"friends" + 0.004*"wealthy" + 0.004*"first" + 0.004*"romance" + 0.004*"family" + 0.

In [23]:
!pip install pyLDAvis

# popular movie topic classification

In [24]:
# !pip install pyLDAvis
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6      0.035887  0.020483       1        1  6.638535
15     0.166171  0.072721       2        1  5.924592
19     0.064145  0.055223       3        1  5.830180
17     0.004497 -0.146070       4        1  5.774598
11     0.028143 -0.006529       5        1  5.567622
8      0.050036 -0.002784       6        1  5.451083
5     -0.138417  0.126006       7        1  5.231225
1     -0.001429  0.027007       8        1  5.139122
12     0.000410 -0.056662       9        1  5.090342
14    -0.028745 -0.064288      10        1  5.074337
3      0.058487 -0.002259      11        1  4.960845
18    -0.022668 -0.071511      12        1  4.857555
13     0.015216 -0.019853      13        1  4.850388
2      0.012252  0.038880      14        1  4.795087
0      0.007296 -0.043050      15        1  4.769599
4     -0.088361 -0.013289      16        1  4.641238
10    -0.019931  0.065165      17        1  4.466934
16    -0.058278 -0.035170      18        1  4.200906
7     -0.027147  0.007301      19        1  3.600307
9     -0.057566  0.048680      20        1  3.135506, topic_info=         Term       Freq      Total Category  logprob  loglift
222      high  20.000000  20.000000  Default  30.0000  30.0000
378      past  22.000000  22.000000  Default  29.0000  29.0000
150    father  19.000000  19.000000  Default  28.0000  28.0000
63       home  30.000000  30.000000  Default  27.0000  27.0000
65       must  30.000000  30.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
27    friends   1.299451  40.688621  Topic20  -5.5138   0.0184
92       gets   1.299451  16.441425  Topic20  -5.5138   0.9245
105     quest   1.299451  12.240626  Topic20  -5.5138   1.2196
1040   deadly   1.299451   6.936982  Topic20  -5.5138   1.7875
899    remote   1.299219  14.260593  Topic20  -5.5140   1.0666

[1316 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3249      2  0.279035  abducts
3249     16  0.558071  abducts
1064      1  0.357015   aboard
1064     19  0.357015   aboard
1403     11  0.698028   abroad
...     ...       ...      ...
15       18  0.017530    young
15       19  0.017530    young
15       20  0.017530    young
2836      5  0.913701     zack
1324     10  0.463760  zealand

[2931 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 16, 20, 18, 12, 9, 6, 2, 13, 15, 4, 19, 14, 3, 1, 5, 11, 17, 8, 10])

In [25]:
!pip install pandas --upgrade

In [26]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6      0.035887  0.020483       1        1  6.638535
15     0.166171  0.072721       2        1  5.924592
19     0.064145  0.055223       3        1  5.830180
17     0.004497 -0.146070       4        1  5.774598
11     0.028143 -0.006529       5        1  5.567622
8      0.050036 -0.002784       6        1  5.451083
5     -0.138417  0.126006       7        1  5.231225
1     -0.001429  0.027007       8        1  5.139122
12     0.000410 -0.056662       9        1  5.090342
14    -0.028745 -0.064288      10        1  5.074337
3      0.058487 -0.002259      11        1  4.960845
18    -0.022668 -0.071511      12        1  4.857555
13     0.015216 -0.019853      13        1  4.850388
2      0.012252  0.038880      14        1  4.795087
0      0.007296 -0.043050      15        1  4.769599
4     -0.088361 -0.013289      16        1  4.641238
10    -0.019931  0.065165      17        1  4.466934
16    -0.058278 -0.035170      18        1  4.200906
7     -0.027147  0.007301      19        1  3.600307
9     -0.057566  0.048680      20        1  3.135506, topic_info=         Term       Freq      Total Category  logprob  loglift
222      high  20.000000  20.000000  Default  30.0000  30.0000
378      past  22.000000  22.000000  Default  29.0000  29.0000
150    father  19.000000  19.000000  Default  28.0000  28.0000
63       home  30.000000  30.000000  Default  27.0000  27.0000
65       must  30.000000  30.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
27    friends   1.299451  40.688621  Topic20  -5.5138   0.0184
92       gets   1.299451  16.441425  Topic20  -5.5138   0.9245
105     quest   1.299451  12.240626  Topic20  -5.5138   1.2196
1040   deadly   1.299451   6.936982  Topic20  -5.5138   1.7875
899    remote   1.299219  14.260593  Topic20  -5.5140   1.0666

[1316 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3249      2  0.279035  abducts
3249     16  0.558071  abducts
1064      1  0.357015   aboard
1064     19  0.357015   aboard
1403     11  0.698028   abroad
...     ...       ...      ...
15       18  0.017530    young
15       19  0.017530    young
15       20  0.017530    young
2836      5  0.913701     zack
1324     10  0.463760  zealand

[2931 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 16, 20, 18, 12, 9, 6, 2, 13, 15, 4, 19, 14, 3, 1, 5, 11, 17, 8, 10])